In [ ]:
"""
Script to automatically set up RO & RW guest collections on the host(s) specified in GLOBUS_HOSTS. A small directory of test files will be
transferred into the RO guest collection
There is logic to prevent the creation of duplicate guest collections, and the standard test files are copied into the RO guest collection.
The script assumes that each endpoint has exactly one storage gateway and exactly one public mapped collection.
Anyone with write permissions to the public mapped collection can run this script.
"""

import globus_sdk
from globus_sdk import TransferClient, TransferAPIError
from globus_sdk.scopes import TransferScopes
from globus_sdk.globus_app import UserApp
from pprint import pprint

########################################################################################################
# USER TO COMPLETE - Enter UserApp Client UUID - UserApp needs to be created first
NATIVE_CLIENT_ID = "xxxx"

# USER TO COMPLETE - Enter Endpoint hostname shown by "globus-connect-server endpoint show" command
# Hostname is found in the "GCS Manager URL" value
GLOBUS_HOSTS = [
    "xxxx.gaccess.io",
]

# USER TO COMPLETE - Enter Globus User UUID for write permissions
GLOBUS_USER_UUID = "xxxx"
########################################################################################################

# AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Public RO Guest Collection
# https://app.globus.org/file-manager?origin_id=9e472d3a-ac18-42d0-bac8-3c9220801fbe&two_pane=true
SRC_COLLECTION = "9e472d3a-ac18-42d0-bac8-3c9220801fbe"
SRC_PATH = "/standard_test_files/5MB-in-tiny-files"
DST_PATH = "/5MB-in-tiny-files"

# # Could also use original files from CERN (slower)
# # https://app.globus.org/file-manager/collections/722751ce-1264-43b8-9160-a9272f746d78
# SRC_COLLECTION = "722751ce-1264-43b8-9160-a9272f746d78"
# SRC_PATH = "/5MB-in-tiny-files"
# DST_PATH = "/5MB-in-tiny-files"


def main():
    user_app = UserApp("create-guest-collections", client_id=NATIVE_CLIENT_ID)
    # pprint(user_app.__dict__)
    for globus_host in GLOBUS_HOSTS:
        gcs_client = globus_sdk.GCSClient(globus_host, app=user_app)
        # pprint(gcs_client.__dict__)
        endpoint_id = gcs_client._endpoint_client_id

        storage_gateways = list(gcs_client.get_storage_gateway_list())
        # pprint(storage_gateways)

        if len(storage_gateways) != 1:
            print(f"Unable to determine unique storage gateway for {globus_info['hostname']}")
            exit(1)

        storage_gateway_id = storage_gateways[0]['id']

        collections = list(gcs_client.get_collection_list())
        # pprint(collections)

        # Find public mapped collection (assumes only one)
        mapped_collections = [collection for collection in collections if collection['collection_type'] == 'mapped' and collection['public']]
        if len(storage_gateways) != 1:
            print(f"Unable to determine unique public mapped collection for {globus_info['hostname']}")
            exit(1)

        mapped_collection = mapped_collections[0]

        transfer_client = TransferClient(app=user_app).add_app_data_access_scope(mapped_collection['id'])
        # pprint(transfer_client.__dict__)

        guest_collections = [
            collection for collection in collections
            if collection['collection_type'] == 'guest'
            and collection['public']
            and collection['mapped_collection_id'] == mapped_collection['id']
            ]

        # pprint(guest_collections)

        # Remove this line if the mapped collection is high assurance
        attach_data_access_scope(gcs_client, mapped_collection['id'])

        ensure_user_credential(gcs_client, storage_gateway_id)

        for collection_type in ['Public RO Guest', 'Public RW Guest']:
            # Create display name and base path - this is a little ugly and should be improved
            display_name = mapped_collection['display_name'].replace('Public Mapped', collection_type)
            collection_base_path = f"/{collection_type.replace(' ', '_')}_Collection"

            try:
                response = transfer_client.operation_mkdir(mapped_collection['id'], collection_base_path)
                # pprint(response)
                print(f'Created directory "{collection_base_path}" in collection "{mapped_collection["display_name"]}"')
            except TransferAPIError as e:
                # pprint(e.__dict__)
                if e.code == 'ExternalError.MkdirFailed.Exists':
                    print(f'Directory "{collection_base_path}" already exists in collection "{mapped_collection["display_name"]}"')
                else:
                    raise e

            collection_request = globus_sdk.GuestCollectionDocument(
                public=True,
                collection_base_path=collection_base_path,
                display_name=display_name,
                mapped_collection_id=mapped_collection['id'],
            )
            # pprint(collection_request)

            found_collections = [
                collection for collection in collections 
                if collection['collection_type'] == 'guest' 
                and collection['public']
                and collection['display_name'] == display_name
                ]

            if found_collections:
                print(f'Public guest collection "{display_name}" already exists.')
                if len(found_collections) == 1:
                    collection = found_collections[0]
                else:
                    print(f'Unable to determine unique match for collection "{display_name}"')
                    exit(1)

                # Unable to update collection_base_path (immutable), so we can't change that
                print(f'Updating existing guest collection "{display_name}" with collection ID {collection["id"]}')
                gcs_client.update_collection(collection['id'], collection_request)

            else:
                collection = gcs_client.create_collection(collection_request)
                print(f'Created new guest collection "{display_name}" with collection ID {collection["id"]}')

            # Add read permissions for all authenticated users on all collections
            rule_data = {
                'DATA_TYPE': 'access',
                'path': '/',
                'permissions': 'r',
                'principal': '',
                'principal_type': 'all_authenticated_users',
            }
            try:
                result = transfer_client.add_endpoint_acl_rule(collection["id"], rule_data)
                # pprint(result)
                print('Added "r" permissions for "all_authenticated_users"')
            except TransferAPIError as e:
                # pprint(e.__dict__)
                if e.code == 'Exists':
                    print('Permissions already exist for "all authenticated users"')
                else:
                    raise e

            # Add read/write permissions for specified user on RW collection
            # Note that this is actually superfluous if the UUID is for the same user who owns the collection
            if ' RW ' in collection_type:
                rule_data = {
                    'DATA_TYPE': 'access',
                    'path': '/',
                    'permissions': 'rw',
                    'principal': GLOBUS_USER_UUID,
                    'principal_type': 'identity',
                }
                try:
                    result = transfer_client.add_endpoint_acl_rule(collection["id"], rule_data)
                    # pprint(result)
                    print(f'Added "rw" permissions for user "{GLOBUS_USER_UUID}"')
                except TransferAPIError as e:
                    # pprint(e.__dict__)
                    if e.code == 'Exists':
                        print(f'Permissions already exist for user "{GLOBUS_USER_UUID}"')
                    else:
                        raise e

            # Transfer standard test files into RO collection
            elif ' RO ' in collection_type:
                try:
                    response = transfer_client.operation_mkdir(collection["id"], DST_PATH) # Error if directory already exists
                    # pprint(response)
                    print(f'Created directory "{DST_PATH}" in collection "{collection["display_name"]}"')

                    transfer_request = globus_sdk.TransferData(
                        source_endpoint=SRC_COLLECTION,
                        destination_endpoint=collection["id"],
                    )
                    transfer_request.add_item(SRC_PATH, DST_PATH)

                    task = transfer_client.submit_transfer(transfer_request)
                    print(f"Submitted transfer of standard test files to {DST_PATH}. Task ID: {task['task_id']}.")
                except TransferAPIError as e:
                    # pprint(e.__dict__)
                    if e.code == 'ExternalError.MkdirFailed.Exists':
                        print(f'Directory "{DST_PATH}" already exists in collection "{collection["display_name"]}"')
                    else:
                        raise e


def attach_data_access_scope(gcs_client, collection_id):
    """Compose and attach a ``data_access`` scope for the supplied collection"""
    endpoint_scopes = gcs_client.get_gcs_endpoint_scopes(gcs_client.endpoint_client_id)
    collection_scopes = gcs_client.get_gcs_collection_scopes(collection_id)

    manage_collections = globus_sdk.Scope(endpoint_scopes.manage_collections)
    data_access = globus_sdk.Scope(collection_scopes.data_access, optional=True)

    manage_collections.add_dependency(data_access)

    gcs_client.add_app_scope(manage_collections)


def ensure_user_credential(gcs_client, storage_gateway_id):
    """
    Ensure that the user has a user credential on the client.
    This is the mapping between Globus Auth (OAuth2) and the local system's permissions.
    """
    # Depending on the endpoint & storage gateway, this request document may need to
    # include more complex information such as a local username.
    # Consult with the endpoint owner for more detailed info on user mappings and
    # other particular requirements.
    req = globus_sdk.UserCredentialDocument(storage_gateway_id=storage_gateway_id)
    try:
        gcs_client.create_user_credential(req)
    except globus_sdk.GCSAPIError as err:
        # A user credential already exists, no need to create it.
        if err.http_status != 409:
            raise

if __name__ == '__main__':
    main()